In [1]:
import os 
os.chdir('/app')

In [2]:
from dotenv import load_dotenv
load_dotenv(dotenv_path='config/.env')
database_path = os.getenv('DATABASE_PATH')
print(database_path)

database/db/ohlcv_data.db


Fetch All Indian Equity Data from API

In [3]:
from data.fetch.indian_equity import fetch_symbol_list_indian_equity
symbols = fetch_symbol_list_indian_equity(index_name='nse_eq_symbols')

In [4]:
from utils.calculation.time import calculate_start_time
start_time = calculate_start_time(timeframe='1y', data_points_back=10)

In [ ]:
from data.gather.indian_equity import gather_ohlcv_indian_equity

symbols , data = gather_ohlcv_indian_equity(timeframe='1d', start_date=start_time)

Store All Indian Equity Data

In [8]:
from finstore.finstore import Finstore

finstore = Finstore(market_name='indian_equity', timeframe='1d', enable_append=True)

In [20]:
finstore.write.symbol_list(data_ohlcv=data)

Writing symbols: 100%|██████████| 1675/1675 [00:09<00:00, 185.88it/s]


Read All Indian Equity Data

In [6]:
from data.fetch.indian_equity import fetch_symbol_list_indian_equity
symbols = fetch_symbol_list_indian_equity(index_name='nse_eq_symbols')

In [ ]:
ohlcv_data = finstore.read.symbol_list(symbol_list=symbols)

In [10]:
ohlcv_data['SBIN.NS']

,timestamp,open,high,low,close,volume,market_name,timeframe
0,2014-10-20 00:00:00,257.700012,259.375000,255.274994,256.945007,233.623901,indian_equity,1d
1,2014-10-21 00:00:00,258.000000,260.700012,256.959991,258.415009,234.960510,indian_equity,1d
2,2014-10-22 00:00:00,260.899994,261.359985,256.510010,257.524994,234.151245,indian_equity,1d
3,2014-10-27 00:00:00,259.200012,261.989990,258.100006,259.779999,236.201569,indian_equity,1d
4,2014-10-28 00:00:00,260.799988,266.429993,260.600006,265.674988,241.561539,indian_equity,1d
...,...,...,...,...,...,...,...,...
2458,2024-10-09 00:00:00,786.700012,804.349976,782.049988,797.400024,797.400024,indian_equity,1d
2459,2024-10-10 00:00:00,799.000000,804.450012,795.000000,797.099976,797.099976,indian_equity,1d
2460,2024-10-11 00:00:00,797.200012,802.700012,793.000000,799.750000,799.750000,indian_equity,1d
2461,2024-10-14 00:00:00,802.000000,809.250000,800.799988,805.150024,805.150024,indian_equity,1d


Write technical indicators for all Indian Equity Data

In [ ]:
from utils.calculation.indicators import calculate_ema
from utils.calculation.supertrend import faster_supertrend
from utils.calculation.slope_r2 import calculate_exponential_regression_optimized
from utils.calculation.optimized_indicators import calculate_spike_optimized, detect_large_gap_optimized, calculate_average_volume_optimized
finstore.write.indicator(ohlcv_data=ohlcv_data, calculation_func=calculate_ema, length=100)
finstore.write.indicator(ohlcv_data=ohlcv_data, calculation_func=calculate_ema, length=200)
finstore.write.indicator(ohlcv_data=ohlcv_data, calculation_func=calculate_ema, length=500)
finstore.write.indicator(ohlcv_data=ohlcv_data, calculation_func=faster_supertrend, period=7, multiplier=3)
finstore.write.indicator(ohlcv_data=ohlcv_data, calculation_func=calculate_exponential_regression_optimized, window=90)
finstore.write.indicator(ohlcv_data=ohlcv_data, calculation_func=calculate_exponential_regression_optimized, window=30)
finstore.write.indicator(ohlcv_data=ohlcv_data, calculation_func=calculate_exponential_regression_optimized, window=15)
finstore.write.indicator(ohlcv_data=ohlcv_data, calculation_func=calculate_spike_optimized, lookback_period=90, spike_threshold=0.15)
finstore.write.indicator(ohlcv_data=ohlcv_data, calculation_func=detect_large_gap_optimized, lookback_period=90, gap_threshold=0.15)
finstore.write.indicator(ohlcv_data=ohlcv_data, calculation_func=calculate_average_volume_optimized, lookback_period=90)

Read ohlcv + technical indicators for all Indian Equity Data

In [11]:
merged_ohlcv_data = finstore.read.symbol_list(symbol_list=symbols, merged_dataframe=True)

In [12]:
merged_ohlcv_data['SBIN.NS'].tail()

,timestamp,open,high,low,close,volume,market_name,timeframe,average_volume_90,ema_100,ema_200,ema_500,gap_90_0.15,slope_r2_product_15,slope_r2_product_30,slope_r2_product_90,spike_90_0.15,supertrend_7_3
2458,2024-10-09 00:00:00,786.700012,804.349976,782.049988,797.400024,797.400024,indian_equity,1d,681779.880396,803.195321,767.703023,671.868918,0.0,-0.000021,-2.065465e-04,-0.000383,0.0,824.143807
2459,2024-10-10 00:00:00,799.000000,804.450012,795.000000,797.099976,797.099976,indian_equity,1d,681178.628053,803.074621,767.995530,672.368842,0.0,-0.000007,-1.383628e-04,-0.000405,0.0,824.143807
2460,2024-10-11 00:00:00,797.200012,802.700012,793.000000,799.750000,799.750000,indian_equity,1d,679171.940228,803.008787,768.311495,672.877350,0.0,-0.000002,-6.446727e-05,-0.000369,0.0,824.143807
2461,2024-10-14 00:00:00,802.000000,809.250000,800.799988,805.150024,805.150024,indian_equity,1d,679697.845816,803.051188,768.678047,673.405384,0.0,-0.000004,-1.394735e-05,-0.000461,0.0,824.143807
2462,2024-10-15 00:00:00,805.500000,807.500000,801.599976,804.650024,804.650024,indian_equity,1d,679961.808030,803.082848,769.035977,673.929315,0.0,0.000003,-5.382415e-08,-0.000539,0.0,824.143807


In [13]:
import pandas as pd

# Assuming ohlcv_data is your dictionary with the structure {symbol: DataFrame}
latest_timestamps = {}

for symbol, df in ohlcv_data.items():
    # Get the latest timestamp for each symbol
    latest_timestamps[symbol] = df['timestamp'].max()

# Convert the dictionary to a pandas Series for easy manipulation
latest_timestamps_series = pd.Series(latest_timestamps)

# Find the minimum of the latest timestamps
min_latest_timestamp = latest_timestamps_series.min()

print("The least of the latest timestamps is:", min_latest_timestamp)

The least of the latest timestamps is: 2024-10-14 00:00:00


In [9]:
symbols, data = gather_ohlcv_indian_equity(timeframe='1d', start_date=pd.to_datetime(min_latest_timestamp), complete_list=False, index_name='nse_eq_symbols')